# RA Yearly Stats

Welcome to RA Yearly Stats! This Jupyter notebook is a window to your yearly RetroAchievements stats. Do you want to know how many games you beat this last year? How many achievements you got? Maybe how they are distributed among the consoles or which dev gave you the most achievements? You're in for a treat!

This is a project I started just for the fun of learning how to make API calls that ended up in what you see now. At some point I was happy enough with the shape it was taking to decide that it might be of interest to people other than me. Ever since I discovered RetroAchievements, I wanted to repay the community in some way for all of the free fun that it has given me, so take this as my little thank you to you all.

## Instructions

In order to be able to show you this data, **RA Yearly Stats just requires your username and your API key**, which you can input in the code cell below. You can find your API key in the RetroAchievements website, under Settings.

Please input them below this text block inside the provided quotation marks for the code to work properly. Then, hit the 'Run All Cells' button that is under 'Run' in the toolbar above and wait a bit for the program to make all the required requests to the RetroAchievements API and work on the data.

As a matter of fact, you can use your API key to not only check your data but also any other users', so feel free to check how other users are doing using your own API key!

## Considerations

- In each section of the showcase, **you will be able to select any year you have RetroAchievements data for** using a selector. The program will default to the last full year which, at the time of uploading the tool, is 2024. If there is no data for that year, it will default to the most recent year you have data of.

- **The program will take some time to load** because it waits a bit between calls to the API to avoid saturating it. Please be patient, users with lots of achievements can take up to a few minutes to process.

- If you **hover your mouse over the data of any graph**, it will show further detail.

- **The program takes only into account hardcore achievement data**. I don't like gatekeeping anybody for playing in softcore mode so my sincere apologies for this decision. The program, as it is now, relies on the achievements being earned in hardcore mode and would not show coherent data if there were softcore achievements mixed in, mainly due to RetroAchievements overwriting softcore achievement data if an achievement is later obtained in hardcore mode. I will try to address this issue in the future because I want everyone to be able to enjoy the tool, so please be patient if you play on softcore mode.

- **The program requires you to have earned at least one hardcore achievement**, it will break otherwise.

- Much of the data shown here is calculated from the achievement data. For example, the "mastery" status of a set is calculated by checking if you have all the achievements and if, between when you mastered a set and now, more achievements have been added and you didn't get them, the program will not consider that you mastered the set. This kind of situations **might cause discrepancies between the data shown at your RetroAchievements profile and the one shown here**. Please take this into consideration when reviewing your data before considering it a bug.

- I'm not a web developer but I tried to use some HTML code to have a better looking display of the information. However, I don't know how the HTML code will behave with any setup different than the one I use. For the record, **the code has been tweaked to work best on Google Chrome in 1920x1080 resolution. Please consider to use a similar setup for best results.**

- I've tried to test the program as thoroughly as possible, but issues are bound to happen once more people try to use the tool. **If you find a bug, please contact me** either through RetroAchievements direct messages or by opening a GitHub issue and explain your issue in as much detail as possible so that I can work on solving it.

In [ ]:
### MODIFY THE CODE BELOW TO INPUT YOUR DATA ###

username = ""
api_key = ""

## DO NOT MODIFY ANY CODE BEYOND THIS POINT ###

In [ ]:
# Here's all the code hidden

import os

os.system("pip install requests numpy pandas matplotlib mplcursors plotly pillow ipywidgets > /dev/null 2>&1")

import requests

import numpy as np
import pandas as pd

import datetime, calendar

import matplotlib.pyplot as plt
import mplcursors

import plotly.graph_objects as go

from PIL import Image
from io import BytesIO
import base64

import ipywidgets as widgets
from IPython.display import display, Markdown, HTML

import RAYearlyStats_backend as RA

def HTML_code_show_picture(picture):

    buffer = BytesIO()
    picture.savefig(buffer, format="png", bbox_inches="tight")
    buffer.seek(0)
    picture_base64 = base64.b64encode(buffer.read()).decode("utf-8")
    buffer.close()

    return f"""<img src="data:image/png;base64,{picture_base64}" style="width: 100%; height: auto; display: block;" alt="Figure">"""

def HTML_draw_horizontal_line():

    display(HTML("""<div style="width: 100%; height: 1px; background-color: black;"></div>"""))

df_historic = RA.retrieve_historic_df(username=username,
                                      api_key=api_key)

df_events = RA.get_event_data(df_historic, drop=True)
    
df_games_data, cheevos_data_dict = RA.retrieve_necessary_games_data(df_historic=df_historic,
                                                                    api_key=api_key)

year_list = list(df_historic["Year"].unique())

default_year = datetime.datetime.now().year - 1
if default_year not in year_list:
    default_year = year_list[-1]

@widgets.interact(df_historic=widgets.fixed(df_historic),
                  year=widgets.Dropdown(
                        options=year_list,
                        value=default_year,
                        description='Year:',
                        disabled=False),
                  df_games_data=widgets.fixed(df_games_data),
                  cheevos_data_dict=widgets.fixed(cheevos_data_dict))
def show_yearly_stats(df_historic, year, df_games_data, cheevos_data_dict):

    ######################
    ### Initialization ###
    ######################

    # Retrieve/calculate necessary data

    user_icon = RA.get_user_icon_fig(username)

    stats     = RA.get_yearly_stats(df_historic, year, df_games_data, cheevos_data_dict)
    dev_stats = RA.get_yearly_favdev_stats(df_historic, year)

    # Parameters

    title_fontsize = 36
    section_title_fontsize = 28
    general_text_fontsize = 18

    # Template for four column displays

    base_html_code = ["""<div style="display: flex; justify-content: space-between; align-items: center; width: 100%; font-family: Arial, sans-serif;">""",
                      """<!-- Column 1 -->""",
                      """<div style="width: 8%; text-align: center;">""",
                      "",
                      """</div>""",
                      """<!-- Column 2 -->""",
                      """<div style="width: 42%; text-align: left;">""",
                      "",
                      """</div>""",
                      """<!-- Column 3 -->""",
                      """<div style="width: 8%; text-align: center;">""",
                      "",
                      """</div>""",
                      """<!-- Column 4 -->""",
                      """<div style="width: 42%; text-align: left;">""",
                      "",
                      """</div>""",
                      """</div>"""]

    # Proper spacing with widget
    
    print()

    #############
    ### Title ###
    #############

    display(HTML(f"""
        <p style="font-size: {title_fontsize}px; margin-bottom: 10px;">
            <b>{username}'s {year} in RetroAchievements</b>
        </p>
    """))

    HTML_draw_horizontal_line()

    ####################
    ### General data ###
    ####################
    
    display(HTML(f"""
        <p style="font-size: 18px; margin-bottom: 10px;">
            Achievements earned: <b>{stats["Achievements total"]}</b>
        </p>
        <p style="font-size: 18px; margin-bottom: 10px;">
            Points earned: <b>{stats["Points total"]} ({stats["RetroPoints total"]})</b>
        </p>
        <p style="font-size: 18px; margin-bottom: 10px;">
            Games with at least one achievement earned: <b>{stats["Game total"]}</b>
        </p>
        <p style="font-size: 18px; margin-bottom: 10px;">
            Beaten games: <b>{len(stats["Beaten games"])}</b>
        </p>
        <p style="font-size: 18px; margin-bottom: 10px;">
            Mastered games: <b>{len(stats["Mastered games"])}</b>
        </p>
    """))

    HTML_draw_horizontal_line()
    
    ####################
    ### Beaten games ###
    ####################

    if len(stats["Beaten games"]) > 0:

        display(HTML(f"""
            <p style="font-size: {section_title_fontsize}px; margin-bottom: 10px;">
                <b>Beaten games</b>
            </p>
        """))

        for i in range(0,len(stats["Beaten games"]),2):

            # First column
    
            game_id = stats["Beaten games"][i]
            game_icon = stats["Game icons"][game_id]
    
            base_html_code[3] = HTML_code_show_picture(game_icon)
            base_html_code[7] = f"""
                <p style="font-size: 16px; margin-bottom: 10px;">{RA.get_game_title(game_id, df_games_data)}</p>
                <p style="font-size: 12px;">{RA.get_game_console(game_id, df_historic)}</p>
            """
    
            # Second column
    
            if i+1 < len(stats["Beaten games"]):
    
                game_id = stats["Beaten games"][i+1]
                game_icon = stats["Game icons"][game_id]
    
                base_html_code[11] = HTML_code_show_picture(game_icon)
                base_html_code[15] = f"""
                    <p style="font-size: 16px; margin-bottom: 10px;">{RA.get_game_title(game_id, df_games_data)}</p>
                    <p style="font-size: 12px;">{RA.get_game_console(game_id, df_historic)}</p>
                """
    
            else:
    
                base_html_code[11] = """&nbsp;"""
                base_html_code[15] = """&nbsp;"""
    
            display(HTML("".join(base_html_code)))
    
        HTML_draw_horizontal_line()

    ######################
    ### Mastered games ###
    ######################

    if len(stats["Mastered games"]) > 0:

        display(HTML(f"""
            <p style="font-size: {section_title_fontsize}px; margin-bottom: 10px;">
                <b>Mastered games</b>
            </p>
        """))

        for i in range(0,len(stats["Mastered games"]),2):

            # First column
    
            game_id = stats["Mastered games"][i]
            game_icon = stats["Game icons"][game_id]
    
            base_html_code[3] = HTML_code_show_picture(game_icon)
            base_html_code[7] = f"""
                <p style="font-size: 16px; margin-bottom: 10px;">{RA.get_game_title(game_id, df_games_data)}</p>
                <p style="font-size: 12px;">{RA.get_game_console(game_id, df_historic)}</p>
            """
    
            # Second column
    
            if i+1 < len(stats["Mastered games"]):
    
                game_id = stats["Mastered games"][i+1]
                game_icon = stats["Game icons"][game_id]
    
                base_html_code[11] = HTML_code_show_picture(game_icon)
                base_html_code[15] = f"""
                    <p style="font-size: 16px; margin-bottom: 10px;">{RA.get_game_title(game_id, df_games_data)}</p>
                    <p style="font-size: 12px;">{RA.get_game_console(game_id, df_historic)}</p>
                """
    
            else:
    
                base_html_code[11] = """&nbsp;"""
                base_html_code[15] = """&nbsp;"""
    
            display(HTML("".join(base_html_code)))
    
        HTML_draw_horizontal_line()

    ################################
    ### Daily point distribution ###
    ################################

    display(HTML(f"""
        <p style="font-size: {section_title_fontsize}px; margin-bottom: 10px;">
            <b>Daily point distribution</b>
        </p>
    """))

    fig = RA.get_figure_daily_points_one_year(df_historic, year)
    fig.show()

    HTML_draw_horizontal_line()

    ###################################################
    ### Achievements with highest RetroPoint value  ###
    ###################################################

    display(HTML(f"""
        <p style="font-size: {section_title_fontsize}px; margin-bottom: 10px;">
            <b>Achievements with highest RetroPoint value</b>
        </p>
    """))

    for i in range(0,len(stats["Hardest achievements"]),2):

        # First column

        achievement = stats["Hardest achievements"][i]
        badge_icon  = stats["Hardest achievements badges"][i]

        base_html_code[3] = HTML_code_show_picture(badge_icon)
        base_html_code[7] = f"""
            <p style="font-size: 16px; margin-bottom: 10px;">{achievement["Title"]} | {achievement["Points"]} ({achievement["TrueRatio"]})</p>
            <p style="font-size: 12px;">{RA.get_game_title(achievement["GameID"], df_games_data)}</p>
        """

        # Second column

        if i+1 < len(stats["Hardest achievements"]):

            achievement = stats["Hardest achievements"][i+1]
            badge_icon = stats["Hardest achievements badges"][i+1]

            base_html_code[11] = HTML_code_show_picture(badge_icon)
            base_html_code[15] = f"""
                <p style="font-size: 16px; margin-bottom: 10px;">{achievement["Title"]} | {achievement["Points"]} ({achievement["TrueRatio"]})</p>
                <p style="font-size: 12px;">{RA.get_game_title(achievement["GameID"], df_games_data)}</p>
            """

        else:

            base_html_code[11] = """&nbsp;"""
            base_html_code[15] = """&nbsp;"""

        display(HTML("".join(base_html_code)))

    HTML_draw_horizontal_line()

    ###########################################
    ### Achievement distribution by console ###
    ###########################################

    display(HTML(f"""
        <p style="font-size: {section_title_fontsize}px; margin-bottom: 10px;">
            <b>Achievement distribution by console</b>
        </p>
    """))

    fig = RA.get_figure_system_distribution(df_historic, year, by="Achievements")
    fig.show()

    HTML_draw_horizontal_line()

    #############################################
    ### Achievement distribution by developer ###
    #############################################

    display(HTML(f"""
        <p style="font-size: {section_title_fontsize}px; margin-bottom: 10px;">
            <b>Achievement distribution by developer</b>
        </p>
    """))

    fig = RA.get_figure_dev_distribution(df_historic, year, by="Achievements")
    fig.show()

    HTML_draw_horizontal_line()

    ##########################
    ### Favorite developer ###
    ##########################

    dev_username = dev_stats["Username"]
    dev_cheevo_total = dev_stats["Achievement total"]
    dev_percent = dev_stats["Achievement %"]

    display(HTML(f"""
        <p style="font-size: {section_title_fontsize}px; margin-bottom: 10px;">
            <b>Favorite developer appreciation corner</b>
        </p>
    """))

    display(HTML(f"""
        <div style="display: flex; justify-content: space-between; align-items: center; width: 100%; font-family: Arial, sans-serif;">
            <!-- Column 1 -->
            <div style="width: 15%; text-align: left;">
                {HTML_code_show_picture(dev_stats["User icon"])}
            </div>
    
            <!-- Column 2 -->
            <div style="width: 85%; text-align: left;">
                <p style="font-size: {section_title_fontsize}px; margin-bottom: 10px;"><b>{dev_username}</b></p>
            </div>
        </div>
    """))

    display(HTML(f"""
        <p style="font-size: {general_text_fontsize}px; margin-bottom: 10px;">
            {dev_username} developed <b>{dev_cheevo_total} achievements</b> of the {stats["Achievements total"]} that you got in {year}. That's the <b>{dev_percent:.2f} %</b>!
        </p>
        <p style="font-size: {general_text_fontsize}px; margin-bottom: 10px;">
            Go thank them for making your {year} more enjoyable!
        </p>
        <p style="font-size: {general_text_fontsize}px; margin-bottom: 10px;">
            Here's a breakdown of the achievements developed by them that you played in {year}:
        </p>
    """))

    for i in range(0,len(dev_stats["Game distribution"]),2):

        # First column

        game_id = dev_stats["Game distribution"].index[i]
        
        if game_id in stats["Game icons"].keys():
            game_icon = stats["Game icons"][game_id]
        else:
            game_icon = RA.get_game_icon_fig(df_historic, game_id)

        base_html_code[3] = HTML_code_show_picture(game_icon)
        base_html_code[7] = f"""
            <p style="font-size: 16px; margin-bottom: 10px;">{RA.get_game_title(game_id, df_games_data)}</p>
            <p style="font-size: 12px;">{RA.get_game_console(game_id, df_historic)}, {dev_stats["Game distribution"].iloc[i]} achievements</p>
        """

        # Second column

        if i+1 < len(dev_stats["Game distribution"]):

            game_id = dev_stats["Game distribution"].index[i+1]
        
            if game_id in stats["Game icons"].keys():
                game_icon = stats["Game icons"][game_id]
            else:
                game_icon = RA.get_game_icon_fig(df_historic, game_id)

            base_html_code[11] = HTML_code_show_picture(game_icon)
            base_html_code[15] = f"""
                <p style="font-size: 16px; margin-bottom: 10px;">{RA.get_game_title(game_id, df_games_data)}</p>
                <p style="font-size: 12px;">{RA.get_game_console(game_id, df_historic)}, {dev_stats["Game distribution"].iloc[i+1]} achievements</p>
            """

        else:

            base_html_code[11] = """&nbsp;"""
            base_html_code[15] = """&nbsp;"""

        display(HTML("".join(base_html_code)))